In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd

browser = Browser('chrome')

In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]"
browser.visit(url)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [3]:
# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)

In [4]:
# This is for loading more products 
max_iteration = 1
iteration = 0

while iteration<= max_iteration:
    # Extract the HTML of the current page
    html = browser.html
    
    # Create a BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')
    
    all_links = soup.find_all("div", class_ = 'coveo-product-items')

    links = []
    
    for link in all_links:
        results = link.a['href']
        links.append(results)
        
    # Find the element by ID
    element = soup.find(id='loadMore')
    
    if element is None:
        # Element not found, exit the loop
        break
        
    # Wait for 2 seconds
    time.sleep(1)
    
    
        
    # Click on the element
    browser.find_by_id('loadMore').click()
    
    
    
    iteration += 1
    
    print(f'Iteration #: {iteration}')
    


Iteration #: 1
Iteration #: 2


In [5]:
len(links)

48

In [6]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_dictionary = {}

max_iteration2 = 4
iteration2 = 0

while iteration2 <= max_iteration2:
    for record in links:
        url = record
        browser.visit(url)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
    #     results = browser.links.find_by_text('English')
    #     results.click()

        old_price = soup.find('span', class_ = 'old-price')
        old = old_price.find('span', class_ = 'price')
        regular_price.append(old.find('span', class_ = 'price').text)

        on_sale_price = soup.find('span', class_ = 'special-price')
        new_price = on_sale_price.find('span', class_ = 'price')
        sale_price.append(new_price.find('span', class_ = 'price').text)

        name.append(soup.find('div', class_ = 'page-title-wrapper').text)
        onsale.append(soup.find('div', class_ = 'amlabel-text').text)
        try:
            rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
        except AttributeError:
            rating.append('N/A')
        reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
        size.append(soup.find('div', class_ = 'lcbo-product-size').text)
        description.append(soup.find('div', class_ = 'testing_note').text)

        categories_all = soup.find('div', class_ = 'breadcrumbs')
        categories = categories_all.find_all('li', class_ = 'item')
        category_list = []

        for category_record in categories:
            category_list.append(category_record.text)
        category = category_list[-2]

        moredetail = soup.find('div', class_ = 'moredetail')
        all_details = moredetail.find_all('li')
        details = {}
        
        for detail_record in all_details:
            details_value = detail_record.find('div', class_ = 'value').text
            details_name = detail_record.find('div', class_ = 'label').text
            details[details_name] = details_value
        
        details_dictionary['details'] = details
        
        iteration2 += 1
        
        print(iteration2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [7]:
# old_price = soup.find('span', class_ = 'old-price')
# old = old_price.find('span', class_ = 'price')
# regular_price = old.find('span', class_ = 'price').text

# sale_price = soup.find('span', class_ = 'special-price')
# new_price = sale_price.find('span', class_ = 'price')
# onsale = new_price.find('span', class_ = 'price').text

# name = soup.find('div', class_ = 'page-title-wrapper').text
# onsale = soup.find('div', class_ = 'amlabel-text').text
# rating = soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text
# reviews = soup.find('div', class_ = 'bv_numReviews_text').text
# size = soup.find('div', class_ = 'lcbo-product-size').text
# description = soup.find('div', class_ = 'testing_note').text

# categories_all = soup.find('div', class_ = 'breadcrumbs')
# categories = categories_all.find_all('li', class_ = 'item')
# category_list = []
# for record in categories:
#     category_list.append(record.text)
# category = category_list[-2]

# moredetail = soup.find('div', class_ = 'moredetail')
# all_details = moredetail.find_all('li')
# len_details = len(all_details)
# details = {}

# for record in all_details:
#     details_value = record.find('div', class_ = 'value').text
#     details_name = record.find('div', class_ = 'label').text
#     details[details_name] = details_value

In [11]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category,
}

# regular_price = [] ok
# sale_price = [] ok
# name = [] ok
# rating = [] ok
# reviews = [] ok
# size = [] ok
# description = [] ok
# category = [] ok 

# details = {}
# # Add the details dictionary to the data dictionary
data.update(details_dictionary)

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

# Display the dataframe
df.head(20)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [9]:
category_list

['Home', 'Products', 'Wine', 'Red Wine', 'Yellow Tail Shiraz']

In [10]:
categories_all = soup.find('div', class_ = 'breadcrumbs')
categories = categories_all.find_all('li', class_ = 'item')

category_list1 = []
    
for category_record in categories:
        category_list1.append(category_record.text)
        category_list1